# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saint george,37.1041,-113.5841,34.95,99,0,5.75,US,2023-02-06 10:49:39
1,1,jiazi,22.8779,116.0670,64.58,90,85,7.11,CN,2023-02-06 10:51:37
2,2,lotofaga,-13.9764,-171.8578,78.58,74,90,9.53,WS,2023-02-06 10:51:37
3,3,agadir,30.4202,-9.5982,61.25,35,4,1.97,MA,2023-02-06 10:51:37
4,4,baruun-urt,46.6806,113.2792,0.30,88,95,2.59,MN,2023-02-06 10:51:38


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp_s = city_data_df["Max Temp"] > 21
max_temp_s = city_data_df["Max Temp"] < 27
wind_speed_s = city_data_df["Wind Speed"] > 5

# Drop any rows with null values
city_temp_range_df = city_data_df[min_temp_s & max_temp_s & wind_speed_s]
city_temp_range_df

# Display sample data
clean_city_temp_range_df = city_temp_range_df.dropna()
clean_city_temp_range_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,korostyshiv,50.3172,29.0563,25.36,74,100,10.76,UA,2023-02-06 10:51:46
141,141,whitehorse,60.7161,-135.0538,23.77,80,75,18.41,CA,2023-02-06 10:52:29
169,169,athabasca,54.7169,-113.2854,25.21,91,48,5.12,CA,2023-02-06 10:52:40
209,209,shelburne,44.0787,-80.2041,23.76,96,90,9.19,CA,2023-02-06 10:52:01
239,239,severo-kurilsk,50.6789,156.1250,24.66,62,99,24.45,RU,2023-02-06 10:53:07
355,355,kostomuksha,64.5710,30.5767,22.96,92,38,5.35,RU,2023-02-06 10:55:11
400,400,peace river,56.2501,-117.2860,23.31,79,20,6.91,CA,2023-02-06 10:55:28
483,483,moosomin,50.1333,-101.6677,26.19,91,100,9.40,CA,2023-02-06 10:55:57
509,509,novikovo,51.1537,37.8879,25.45,82,100,10.49,RU,2023-02-06 10:56:06
544,544,platteville,42.7342,-90.4785,23.23,86,0,8.05,US,2023-02-06 10:56:20


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_temp_range_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
27,27,korostyshiv,50.3172,29.0563,25.36,74,100,10.76,UA,2023-02-06 10:51:46,
141,141,whitehorse,60.7161,-135.0538,23.77,80,75,18.41,CA,2023-02-06 10:52:29,
169,169,athabasca,54.7169,-113.2854,25.21,91,48,5.12,CA,2023-02-06 10:52:40,
209,209,shelburne,44.0787,-80.2041,23.76,96,90,9.19,CA,2023-02-06 10:52:01,
239,239,severo-kurilsk,50.6789,156.1250,24.66,62,99,24.45,RU,2023-02-06 10:53:07,
355,355,kostomuksha,64.5710,30.5767,22.96,92,38,5.35,RU,2023-02-06 10:55:11,
400,400,peace river,56.2501,-117.2860,23.31,79,20,6.91,CA,2023-02-06 10:55:28,
483,483,moosomin,50.1333,-101.6677,26.19,91,100,9.40,CA,2023-02-06 10:55:57,
509,509,novikovo,51.1537,37.8879,25.45,82,100,10.49,RU,2023-02-06 10:56:06,
544,544,platteville,42.7342,-90.4785,23.23,86,0,8.05,US,2023-02-06 10:56:20,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = hotel_df.loc[index, "Lng"] 
    lat = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    print(name_address.url)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
https://api.geoapify.com/v2/places?apiKey=093076ae61994759970d94a6cf11f719&filter=circle%3A29.0563%2C50.3172%2C10000&bias=proximity%3A29.0563%2C50.3172
korostyshiv - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?apiKey=093076ae61994759970d94a6cf11f719&filter=circle%3A-135.0538%2C60.7161%2C10000&bias=proximity%3A-135.0538%2C60.7161
whitehorse - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?apiKey=093076ae61994759970d94a6cf11f719&filter=circle%3A-113.2854%2C54.7169%2C10000&bias=proximity%3A-113.2854%2C54.7169
athabasca - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?apiKey=093076ae61994759970d94a6cf11f719&filter=circle%3A-80.2041%2C44.0787%2C10000&bias=proximity%3A-80.2041%2C44.0787
shelburne - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?apiKey=093076ae61994759970d94a6cf11f719&filter=circle%3A156.125%2C50.6789%2C10000&bias=proximity%3A156.125%2C50.6789
severo-kurilsk - nearest hotel: No ho

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
27,27,korostyshiv,50.3172,29.0563,25.36,74,100,10.76,UA,2023-02-06 10:51:46,No hotel found
141,141,whitehorse,60.7161,-135.0538,23.77,80,75,18.41,CA,2023-02-06 10:52:29,No hotel found
169,169,athabasca,54.7169,-113.2854,25.21,91,48,5.12,CA,2023-02-06 10:52:40,No hotel found
209,209,shelburne,44.0787,-80.2041,23.76,96,90,9.19,CA,2023-02-06 10:52:01,No hotel found
239,239,severo-kurilsk,50.6789,156.1250,24.66,62,99,24.45,RU,2023-02-06 10:53:07,No hotel found
355,355,kostomuksha,64.5710,30.5767,22.96,92,38,5.35,RU,2023-02-06 10:55:11,No hotel found
400,400,peace river,56.2501,-117.2860,23.31,79,20,6.91,CA,2023-02-06 10:55:28,No hotel found
483,483,moosomin,50.1333,-101.6677,26.19,91,100,9.40,CA,2023-02-06 10:55:57,No hotel found
509,509,novikovo,51.1537,37.8879,25.45,82,100,10.49,RU,2023-02-06 10:56:06,No hotel found
544,544,platteville,42.7342,-90.4785,23.23,86,0,8.05,US,2023-02-06 10:56:20,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City"
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)